In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import datetime

from datetime import timedelta
from shapely.geometry import Point, LineString, Polygon

In [2]:
data = pd.read_csv("Taxi_Trips_-_2021.csv")

In [3]:
cols = ['Trip Start Timestamp', 'Trip End Timestamp',
       'Trip Seconds', 'Trip Miles', 'Pickup Community Area',
       'Dropoff Community Area', 'Tips',
       'Trip Total','Pickup Centroid Latitude',
       'Pickup Centroid Longitude', 'Pickup Centroid Location']

In [4]:
dt = pd.DataFrame(data,columns=cols)

In [5]:
dt.info

<bound method DataFrame.info of            Trip Start Timestamp      Trip End Timestamp  Trip Seconds  \
0        01/01/2021 12:00:00 AM  01/01/2021 12:15:00 AM        1021.0   
1        01/01/2021 12:00:00 AM  01/01/2021 03:30:00 PM       56047.0   
2        01/01/2021 12:00:00 AM  01/01/2021 12:00:00 AM         120.0   
3        01/01/2021 12:00:00 AM  01/01/2021 12:00:00 AM           0.0   
4        01/01/2021 12:00:00 AM  01/01/2021 12:00:00 AM          50.0   
...                         ...                     ...           ...   
3948040  12/31/2021 11:45:00 PM  01/01/2022 12:00:00 AM         900.0   
3948041  12/31/2021 11:45:00 PM  01/01/2022 12:00:00 AM         805.0   
3948042  12/31/2021 11:45:00 PM  01/01/2022 12:30:00 AM        3027.0   
3948043  12/31/2021 11:45:00 PM  01/01/2022 12:15:00 AM        2400.0   
3948044  12/31/2021 11:45:00 PM  01/01/2022 01:00:00 AM        4620.0   

         Trip Miles  Pickup Community Area  Dropoff Community Area   Tips  \
0             

In [6]:
dt = dt[dt['Trip Miles']>0]
dt = dt[dt['Trip Total']>0]
dt = dt[dt['Trip Start Timestamp'] > '01/01/2021 00:00:00']
dt = dt[dt['Trip Start Timestamp'] < '12/31/2021 23:45:00']
dt = dt.drop(dt[(dt['Trip Start Timestamp'] > dt['Trip End Timestamp'])].index).reset_index(drop=True)
dt = dt.drop(dt[(dt['Pickup Centroid Longitude'] == np.nan)].index).reset_index(drop=True)
dt = dt.drop(dt[(dt['Pickup Centroid Latitude'] == np.nan)].index).reset_index(drop=True)
dt = dt.drop(dt[(dt['Pickup Community Area'] == np.nan)].index).reset_index(drop=True)
dt = dt.drop([0]).reset_index(drop=True)

In [7]:
dt.head(30)

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Community Area,Dropoff Community Area,Tips,Trip Total,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location
0,01/01/2021 12:00:00 AM,01/01/2021 12:00:00 AM,120.0,0.10,8.0,8.0,0.00,4.50,41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
1,01/01/2021 12:00:00 AM,01/01/2021 12:00:00 AM,50.0,0.38,60.0,31.0,0.00,4.00,41.836150,-87.648788,POINT (-87.6487879519 41.8361501547)
2,01/01/2021 12:00:00 AM,01/01/2021 12:15:00 AM,960.0,10.60,33.0,NaN,0.00,28.00,41.857184,-87.620335,POINT (-87.6203346241 41.8571838585)
3,01/01/2021 12:00:00 AM,01/01/2021 12:15:00 AM,900.0,0.30,28.0,22.0,0.00,18.25,41.874005,-87.663518,POINT (-87.6635175498 41.874005383)
4,01/01/2021 12:00:00 AM,01/01/2021 12:15:00 AM,1140.0,4.30,6.0,4.0,0.00,15.25,41.944227,-87.655998,POINT (-87.6559981815 41.9442266014)
5,01/01/2021 12:00:00 AM,01/01/2021 12:30:00 AM,1800.0,9.80,8.0,68.0,0.00,29.00,41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
6,01/01/2021 12:00:00 AM,01/01/2021 12:00:00 AM,240.0,0.10,76.0,76.0,0.00,10.75,41.980264,-87.913625,POINT (-87.913624596 41.9802643146)
7,01/01/2021 12:00:00 AM,01/01/2021 12:15:00 AM,1620.0,7.10,70.0,42.0,0.00,22.25,41.745758,-87.708366,POINT (-87.7083657043 41.7457577128)
8,01/01/2021 12:00:00 AM,01/01/2021 12:00:00 AM,360.0,2.10,56.0,56.0,0.00,12.25,41.792592,-87.769615,POINT (-87.7696154528 41.7925923603)
9,01/01/2021 12:00:00 AM,01/01/2021 12:30:00 AM,1260.0,9.20,56.0,75.0,5.08,34.33,41.792592,-87.769615,POINT (-87.7696154528 41.7925923603)


In [10]:
new_dt=dt.rename(columns = {'Pickup Community Area':'area_numbe'})
new_dt=pd.DataFrame(new_dt,columns=['Trip Start Timestamp','area_numbe'])
new_dt=new_dt.fillna(0)
new_dt['area_numbe'] = new_dt['area_numbe'].astype(int)

In [11]:
new_dt=new_dt.groupby(new_dt['area_numbe']).count()

In [60]:
new_dt.head(30)

,count
area_numbe,
1,27983
2,33368
3,59528
4,18302
5,10220
6,116421
7,69726
8,731084
9,1180


In [13]:
new_dt = pd.DataFrame(new_dt,columns=['Trip Start Timestamp'])
new_dt.columns = ['count']

In [14]:
new_dt = new_dt.drop(labels=0, axis=0)

In [15]:
new_dt.head()

,count
area_numbe,
1,27983
2,33368
3,59528
4,18302
5,10220


In [23]:
new_dt.to_csv(r'C:\Users\95174\OneDrive\文档\CS424\taxi_count.csv')

In [48]:
chicago = gpd.read_file('chicago_area.geojson')

In [49]:
chicago.head()

,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,geometry
0,DOUGLAS,0,46004621.1581,0,35,35,0,0,31027.0545098,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ..."
1,OAKLAND,0,16913961.0408,0,36,36,0,0,19565.5061533,"MULTIPOLYGON (((-87.59215 41.81693, -87.59231 ..."
2,FULLER PARK,0,19916704.8692,0,37,37,0,0,25339.0897503,"MULTIPOLYGON (((-87.62880 41.80189, -87.62879 ..."
3,GRAND BOULEVARD,0,48492503.1554,0,38,38,0,0,28196.8371573,"MULTIPOLYGON (((-87.60671 41.81681, -87.60670 ..."
4,KENWOOD,0,29071741.9283,0,39,39,0,0,23325.1679062,"MULTIPOLYGON (((-87.59215 41.81693, -87.59215 ..."


In [50]:
chicago= chicago.sort_values('area_numbe')
#chicago['area_numbe']=chicago['area_numbe'].astype(int)
chicago=chicago.set_index('area_numbe')

In [51]:
chicago.head()

,community,area,shape_area,perimeter,area_num_1,comarea_id,comarea,shape_len,geometry
area_numbe,,,,,,,,,
1,ROGERS PARK,0,51259902.4506,0,1,0,0,34052.3975757,"MULTIPOLYGON (((-87.65456 41.99817, -87.65574 ..."
10,NORWOOD PARK,0,121959105.47,0,10,0,0,80368.3743778,"MULTIPOLYGON (((-87.78002 41.99741, -87.78049 ..."
11,JEFFERSON PARK,0,64868161.6818,0,11,0,0,44011.9571704,"MULTIPOLYGON (((-87.75264 41.96797, -87.75279 ..."
12,FOREST GLEN,0,89130893.6364,0,12,0,0,74493.821604,"MULTIPOLYGON (((-87.76919 42.00489, -87.76919 ..."
13,NORTH PARK,0,70288706.4343,0,13,0,0,41581.9486539,"MULTIPOLYGON (((-87.70690 41.98308, -87.70688 ..."


In [54]:
num = chicago['area_num_1']
num = num.astype(int)

<bound method Series.info of area_numbe
1      1
10    10
11    11
12    12
13    13
      ..
75    75
76    76
77    77
8      8
9      9
Name: area_num_1, Length: 77, dtype: int32>

In [61]:
n_dt = num.to_frame().merge(new_dt, left_on = 'area_num_1', right_on='area_numbe')
n_dt.head()

,area_num_1,count
0,1,27983
1,10,9587
2,11,8981
3,12,5517
4,13,6857


In [70]:
con_dt = pd.concat([n_dt, chicago])
#pd.concat([n_dt, chicago],axis=1)

In [71]:
con_dt.head(30)

,area_num_1,count,community,area,shape_area,perimeter,comarea_id,comarea,shape_len,geometry
0,1,27983.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,10,9587.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,11,8981.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,12,5517.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,13,6857.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5,14,10869.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
6,15,8109.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7,16,16904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
8,17,1965.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9,18,651.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [28]:
merge_dt = gpd.merge(new_dt,chicago, right_on='area_numbe', left_index=True)
merge_dt = gpd.DataFrame(merge_dt,columns=['community', 'shape_area',
       'shape_len', 'count', 'geometry'])

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [19]:
merge_dt.head()

,community,shape_area,shape_len,count,geometry
area_numbe,,,,,
1,ROGERS PARK,51259902.4506,34052.3975757,27983,"MULTIPOLYGON (((-87.65456 41.99817, -87.65574 ..."
2,WEST RIDGE,98429094.8621,43020.6894583,33368,"MULTIPOLYGON (((-87.68465 42.01948, -87.68464 ..."
3,UPTOWN,65095642.7289,46972.7945549,59528,"MULTIPOLYGON (((-87.64102 41.95480, -87.64400 ..."
4,LINCOLN SQUARE,71352328.2399,36624.6030848,18302,"MULTIPOLYGON (((-87.67441 41.97610, -87.67440 ..."
5,NORTH CENTER,57054167.85,31391.6697542,10220,"MULTIPOLYGON (((-87.67336 41.93234, -87.67342 ..."


In [20]:
merge_dt = gpd.GeoDataFrame(merge_dt, geometry=merge_dt['geometry'], crs=4326)

In [21]:
merge_dt.to_json()
#merge_dt.dtypes
merge_dt.to_csv(r'C:\Users\95174\OneDrive\文档\CS424\chicago_data.csv')